**WEB APP**

In [ ]:
!pip install streamlit pyngrok pycryptodome pqcrypto

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("32fY3gBzJBcikmicjFisGerigCJ_4vuhXvMR58udmz6n1t3pF")

In [ ]:
%%writefile app.py
import time, json, hashlib, hmac, base64
import streamlit as st
from Crypto.Cipher import ChaCha20, AES
from Crypto.Random import get_random_bytes
from pqcrypto.kem import ml_kem_512

# -------------------------
# Utility Functions
# -------------------------
def sha256(data: bytes):
    return hashlib.sha256(data).digest()

def sha256_hex(data: bytes):
    return hashlib.sha256(data).hexdigest()

def hmac_sha256(key, data):
    return hmac.new(key, data, hashlib.sha256).digest()

def b64e(b: bytes) -> str:
    return base64.b64encode(b).decode()

def b64d(s: str) -> bytes:
    return base64.b64decode(s.encode())

class BlockCipherAES:
    def __init__(self, key_bytes):
        self.key = key_bytes[:16]  # AES-128
        self.block_size = 16

    def pad(self, data):
        pad_len = self.block_size - (len(data) % self.block_size)
        return data + bytes([pad_len]) * pad_len

    def unpad(self, data):
        pad_len = data[-1]
        return data[:-pad_len]

    def encrypt(self, plaintext):
        cipher = AES.new(self.key, AES.MODE_CBC)
        ct = cipher.encrypt(self.pad(plaintext))
        return cipher.iv + ct

    def decrypt(self, ciphertext):
        iv = ciphertext[:16]
        ct = ciphertext[16:]
        cipher = AES.new(self.key, AES.MODE_CBC, iv=iv)
        pt = cipher.decrypt(ct)
        return self.unpad(pt)

# -------------------------
# Hybrid Encryption
# -------------------------
def hybrid_encrypt(plaintext_bytes):
    aes_key = get_random_bytes(32)
    chacha_key = get_random_bytes(32)
    mac_key = get_random_bytes(32)

    # AES
    block_cipher = BlockCipherAES(aes_key)
    aes_ct = block_cipher.encrypt(plaintext_bytes)

    # ChaCha20
    cipher_chacha = ChaCha20.new(key=chacha_key)
    chacha_ct = cipher_chacha.nonce + cipher_chacha.encrypt(plaintext_bytes)

    # PQC: Kyber (ML-KEM)
    pk, sk = ml_kem_512.generate_keypair()
    ct_kem, ss_enc = ml_kem_512.encrypt(pk)
    ss_dec = ml_kem_512.decrypt(sk, ct_kem)

    wrap_key = sha256(ss_enc)
    wrapped = hmac_sha256(wrap_key, aes_key + chacha_key + mac_key)

    bundle = aes_ct + chacha_ct + ct_kem + wrapped
    tag = hmac_sha256(mac_key, bundle)

    package = {
        'aes_ct': b64e(aes_ct),
        'chacha_ct': b64e(chacha_ct),
        'kem_ct': b64e(ct_kem),
        'wrapped': b64e(wrapped),
        'tag': b64e(tag),
        'pk': b64e(pk),
        'sk': b64e(sk),
        'aes_key': b64e(aes_key),
        'chacha_key': b64e(chacha_key),
        'mac_key': b64e(mac_key)
    }
    return package

def hybrid_decrypt(package):
    aes_ct = b64d(package['aes_ct'])
    chacha_ct = b64d(package['chacha_ct'])
    ct_kem = b64d(package['kem_ct'])
    wrapped = b64d(package['wrapped'])
    tag = b64d(package['tag'])

    pk = b64d(package['pk'])
    sk = b64d(package['sk'])
    aes_key = b64d(package['aes_key'])
    chacha_key = b64d(package['chacha_key'])
    mac_key = b64d(package['mac_key'])

    ss_dec = ml_kem_512.decrypt(sk, ct_kem)
    wrap_key = sha256(ss_dec)

    expected_wrap = hmac_sha256(wrap_key, aes_key + chacha_key + mac_key)
    if expected_wrap != wrapped:
        raise ValueError("Wrapped key verification failed")

    bundle = aes_ct + chacha_ct + ct_kem + wrapped
    if not hmac.compare_digest(hmac_sha256(mac_key, bundle), tag):
        raise ValueError("HMAC verification failed")

    # AES
    block_cipher = BlockCipherAES(aes_key)
    aes_pt = block_cipher.decrypt(aes_ct)

    # ChaCha20
    nonce = chacha_ct[:8]
    ct_body = chacha_ct[8:]
    cipher_chacha = ChaCha20.new(key=chacha_key, nonce=nonce)
    chacha_pt = cipher_chacha.decrypt(ct_body)

    return aes_pt, chacha_pt

# -------------------------
# Streamlit UI
# -------------------------
st.sidebar.title("🔐 Hybrid Crypto Demo")
mode = st.sidebar.radio("Choose Mode:", ["💳 Transaction Simulation", "🛠️ Custom Encrypt/Decrypt Tool"])

# -------------------------
# Mode 1: Transaction Simulation
# -------------------------
if mode == "💳 Transaction Simulation":
    st.title("💳 Digital Transaction Security Demo")
    st.write("""
    This simulates what happens in the **background of digital payments**
    (like UPI or net banking).
    A transaction JSON is encrypted with **AES + ChaCha20 + Kyber512 + HMAC**
    and then decrypted back.
    """)

    if st.button("Simulate Transaction"):
        # Example JSON transaction
        transaction = {
            "from": "Alice",
            "to": "Bob",
            "amount": 500,
            "currency": "INR",
            "timestamp": time.ctime()
        }
        st.subheader("📑 Transaction JSON (Original)")
        st.json(transaction)

        # Encrypt
        message = json.dumps(transaction).encode()
        package = hybrid_encrypt(message)

        st.subheader("🔒 Encrypted Transaction (Stored as JSON)")
        st.json(package)

        # Decrypt
        aes_pt, chacha_pt = hybrid_decrypt(package)
        recovered = json.loads(aes_pt.decode())

        st.subheader("🔓 Decrypted Transaction JSON")
        st.json(recovered)

        st.success("✅ This shows how your online payments are secured internally!")

# -------------------------
# Mode 2: Custom Encrypt/Decrypt Tool
# -------------------------
else:
    st.title("🛠️ Hybrid Cryptography Tool")
    st.write("AES + ChaCha20 + Post-Quantum Kyber (ML-KEM) + HMAC")

    # Encryption Section
    st.header("Encryption")
    msg = st.text_area("Enter a message to encrypt:", "Hello Hybrid World!")
    if st.button("Encrypt Message"):
        package = hybrid_encrypt(msg.encode())
        st.success("Message Encrypted Successfully ✅")
        st.json(package)

        # Download JSON
        st.download_button(
            label="⬇️ Download Encrypted Package",
            data=json.dumps(package, indent=2),
            file_name="ciphertext_package.json",
            mime="application/json"
        )

    # Decryption Section
    st.header("Decryption")
    uploaded_file = st.file_uploader("Upload a ciphertext package (JSON)", type="json")
    if uploaded_file:
        package = json.load(uploaded_file)
        try:
            aes_pt, chacha_pt = hybrid_decrypt(package)
            st.success("✅ Decryption Successful")
            st.write("AES Recovered:", aes_pt.decode(errors="ignore"))
            st.write("ChaCha20 Recovered:", chacha_pt.decode(errors="ignore"))

            if st.button("🔍 Show Hashes"):
                st.code(f"""
Original plaintext SHA-256:   {sha256_hex(aes_pt)}
AES recovered SHA-256:       {sha256_hex(aes_pt)}
ChaCha20 recovered SHA-256:  {sha256_hex(chacha_pt)}
AES ciphertext SHA-256:      {sha256_hex(b64d(package['aes_ct']))}
ChaCha20 ciphertext SHA-256: {sha256_hex(b64d(package['chacha_ct']))}
""")
        except Exception as e:
            st.error(f"❌ Decryption Failed: {e}")


Overwriting app.py


In [ ]:
public_url = ngrok.connect(8501)
print("Streamlit app running at:", public_url)
!streamlit run app.py --server.port 8501